In [1]:
!pip install pyspark


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 44 kB/s 
     |████████████████████████████████| 199 kB 49.0 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=daac84d234ee64a2ca8ff9df6565408bf1f2ed69699e4ba5bc96ed5a1d027515
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [6]:
import pyspark
import pandas as pd
import numpy as np
from datetime import date,timedelta, datetime

In [10]:
from pyspark.sql import SparkSession, SQLContext

from pyspark.sql.functions import *
from pyspark.sql.types import *


In [13]:
sc = SparkSession.builder.appName('Pyspark').config("spark.sql.shuffle.partitions","50").config("spark.driver.maxResults","5g").getOrCreate()

In [14]:
sc

In [16]:
#reading json file
dataframe = sc.read.json('nyt2.json')

In [17]:
# Look at the data

dataframe.show(10)

+--------------------+--------------------+--------------------+-----------------+--------------------+-------------+-----------------+-------------+----+--------------+--------------------+-------------+
|                 _id|  amazon_product_url|              author| bestsellers_date|         description|        price|   published_date|    publisher|rank|rank_last_week|               title|weeks_on_list|
+--------------------+--------------------+--------------------+-----------------+--------------------+-------------+-----------------+-------------+----+--------------+--------------------+-------------+
|{5b4aa4ead3089013...|http://www.amazon...|       Dean R Koontz|{{1211587200000}}|Odd Thomas, who c...|   {null, 27}|{{1212883200000}}|       Bantam| {1}|           {0}|           ODD HOURS|          {1}|
|{5b4aa4ead3089013...|http://www.amazon...|     Stephenie Meyer|{{1211587200000}}|Aliens have taken...|{25.99, null}|{{1212883200000}}|Little, Brown| {2}|           {1}|           

In [18]:
#simple data inspection

In [19]:
#Returns the columnname and Datatypes

dataframe.dtypes

[('_id', 'struct<$oid:string>'),
 ('amazon_product_url', 'string'),
 ('author', 'string'),
 ('bestsellers_date', 'struct<$date:struct<$numberLong:string>>'),
 ('description', 'string'),
 ('price', 'struct<$numberDouble:string,$numberInt:string>'),
 ('published_date', 'struct<$date:struct<$numberLong:string>>'),
 ('publisher', 'string'),
 ('rank', 'struct<$numberInt:string>'),
 ('rank_last_week', 'struct<$numberInt:string>'),
 ('title', 'string'),
 ('weeks_on_list', 'struct<$numberInt:string>')]

In [20]:
#display whole data

dataframe.show()

+--------------------+--------------------+--------------------+-----------------+--------------------+-------------+-----------------+--------------------+----+--------------+--------------------+-------------+
|                 _id|  amazon_product_url|              author| bestsellers_date|         description|        price|   published_date|           publisher|rank|rank_last_week|               title|weeks_on_list|
+--------------------+--------------------+--------------------+-----------------+--------------------+-------------+-----------------+--------------------+----+--------------+--------------------+-------------+
|{5b4aa4ead3089013...|http://www.amazon...|       Dean R Koontz|{{1211587200000}}|Odd Thomas, who c...|   {null, 27}|{{1212883200000}}|              Bantam| {1}|           {0}|           ODD HOURS|          {1}|
|{5b4aa4ead3089013...|http://www.amazon...|     Stephenie Meyer|{{1211587200000}}|Aliens have taken...|{25.99, null}|{{1212883200000}}|       Little, Br

In [21]:
#Returns first n rows

dataframe.head()

Row(_id=Row($oid='5b4aa4ead3089013507db18b'), amazon_product_url='http://www.amazon.com/Odd-Hours-Dean-Koontz/dp/0553807056?tag=NYTBS-20', author='Dean R Koontz', bestsellers_date=Row($date=Row($numberLong='1211587200000')), description='Odd Thomas, who can communicate with the dead, confronts evil forces in a California coastal town.', price=Row($numberDouble=None, $numberInt='27'), published_date=Row($date=Row($numberLong='1212883200000')), publisher='Bantam', rank=Row($numberInt='1'), rank_last_week=Row($numberInt='0'), title='ODD HOURS', weeks_on_list=Row($numberInt='1'))

In [24]:
#Returns first Row

dataframe.first()

Row(_id=Row($oid='5b4aa4ead3089013507db18b'), amazon_product_url='http://www.amazon.com/Odd-Hours-Dean-Koontz/dp/0553807056?tag=NYTBS-20', author='Dean R Koontz', bestsellers_date=Row($date=Row($numberLong='1211587200000')), description='Odd Thomas, who can communicate with the dead, confronts evil forces in a California coastal town.', price=Row($numberDouble=None, $numberInt='27'), published_date=Row($date=Row($numberLong='1212883200000')), publisher='Bantam', rank=Row($numberInt='1'), rank_last_week=Row($numberInt='0'), title='ODD HOURS', weeks_on_list=Row($numberInt='1'))

In [26]:
#Returns frist n rows

dataframe.take(5)

[Row(_id=Row($oid='5b4aa4ead3089013507db18b'), amazon_product_url='http://www.amazon.com/Odd-Hours-Dean-Koontz/dp/0553807056?tag=NYTBS-20', author='Dean R Koontz', bestsellers_date=Row($date=Row($numberLong='1211587200000')), description='Odd Thomas, who can communicate with the dead, confronts evil forces in a California coastal town.', price=Row($numberDouble=None, $numberInt='27'), published_date=Row($date=Row($numberLong='1212883200000')), publisher='Bantam', rank=Row($numberInt='1'), rank_last_week=Row($numberInt='0'), title='ODD HOURS', weeks_on_list=Row($numberInt='1')),
 Row(_id=Row($oid='5b4aa4ead3089013507db18c'), amazon_product_url='http://www.amazon.com/The-Host-Novel-Stephenie-Meyer/dp/0316218502?tag=NYTBS-20', author='Stephenie Meyer', bestsellers_date=Row($date=Row($numberLong='1211587200000')), description='Aliens have taken control of the minds and bodies of most humans, but one woman won’t surrender.', price=Row($numberDouble='25.99', $numberInt=None), published_date=

In [27]:
#display statistics

dataframe.describe().show()

+-------+--------------------+---------------+--------------------+---------+------------------+
|summary|  amazon_product_url|         author|         description|publisher|             title|
+-------+--------------------+---------------+--------------------+---------+------------------+
|  count|               10195|          10195|               10195|    10195|             10195|
|   mean|                null|           null|                null|     null|1877.7142857142858|
| stddev|                null|           null|                null|     null| 370.9760613506458|
|    min|http://www.amazon...|        AJ Finn|                    |      ACE|  10TH ANNIVERSARY|
|    max|https://www.amazo...|various authors|’Tis for the Rebe...|allantine|               ZOO|
+-------+--------------------+---------------+--------------------+---------+------------------+



In [29]:
#returns columns as a list

dataframe.columns

['_id',
 'amazon_product_url',
 'author',
 'bestsellers_date',
 'description',
 'price',
 'published_date',
 'publisher',
 'rank',
 'rank_last_week',
 'title',
 'weeks_on_list']

In [30]:
#Count the number of records

dataframe.count()

10195

In [31]:
#count the number of records excluding duplicates

dataframe.distinct().count()

10195

In [32]:
#print physical plans 

dataframe.explain('simple')

== Physical Plan ==
FileScan json [_id#40,amazon_product_url#41,author#42,bestsellers_date#43,description#44,price#45,published_date#46,publisher#47,rank#48,rank_last_week#49,title#50,weeks_on_list#51] Batched: false, DataFilters: [], Format: JSON, Location: InMemoryFileIndex(1 paths)[file:/content/nyt2.json], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<_id:struct<$oid:string>,amazon_product_url:string,author:string,bestsellers_date:struct<$d...




In [34]:
#prints all the plans

dataframe.explain('extended')

== Parsed Logical Plan ==
Relation [_id#40,amazon_product_url#41,author#42,bestsellers_date#43,description#44,price#45,published_date#46,publisher#47,rank#48,rank_last_week#49,title#50,weeks_on_list#51] json

== Analyzed Logical Plan ==
_id: struct<$oid:string>, amazon_product_url: string, author: string, bestsellers_date: struct<$date:struct<$numberLong:string>>, description: string, price: struct<$numberDouble:string,$numberInt:string>, published_date: struct<$date:struct<$numberLong:string>>, publisher: string, rank: struct<$numberInt:string>, rank_last_week: struct<$numberInt:string>, title: string, weeks_on_list: struct<$numberInt:string>
Relation [_id#40,amazon_product_url#41,author#42,bestsellers_date#43,description#44,price#45,published_date#46,publisher#47,rank#48,rank_last_week#49,title#50,weeks_on_list#51] json

== Optimized Logical Plan ==
Relation [_id#40,amazon_product_url#41,author#42,bestsellers_date#43,description#44,price#45,published_date#46,publisher#47,rank#48,rank

In [35]:
dataframe.explain('codegen')

Found 0 WholeStageCodegen subtrees.



In [36]:
dataframe.explain('cost')

== Optimized Logical Plan ==
Relation [_id#40,amazon_product_url#41,author#42,bestsellers_date#43,description#44,price#45,published_date#46,publisher#47,rank#48,rank_last_week#49,title#50,weeks_on_list#51] json, Statistics(sizeInBytes=5.8 MiB)

== Physical Plan ==
FileScan json [_id#40,amazon_product_url#41,author#42,bestsellers_date#43,description#44,price#45,published_date#46,publisher#47,rank#48,rank_last_week#49,title#50,weeks_on_list#51] Batched: false, DataFilters: [], Format: JSON, Location: InMemoryFileIndex(1 paths)[file:/content/nyt2.json], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<_id:struct<$oid:string>,amazon_product_url:string,author:string,bestsellers_date:struct<$d...




In [37]:
dataframe.explain('formatted')

== Physical Plan ==
Scan json  (1)


(1) Scan json 
Output [12]: [_id#40, amazon_product_url#41, author#42, bestsellers_date#43, description#44, price#45, published_date#46, publisher#47, rank#48, rank_last_week#49, title#50, weeks_on_list#51]
Batched: false
Location: InMemoryFileIndex [file:/content/nyt2.json]
ReadSchema: struct<_id:struct<$oid:string>,amazon_product_url:string,author:string,bestsellers_date:struct<$date:struct<$numberLong:string>>,description:string,price:struct<$numberDouble:string,$numberInt:string>,published_date:struct<$date:struct<$numberLong:string>>,publisher:string,rank:struct<$numberInt:string>,rank_last_week:struct<$numberInt:string>,title:string,weeks_on_list:struct<$numberInt:string>>




In [39]:
# Common functions

#Removing the duplicates

dataframe_dropdup = dataframe.dropDuplicates()

In [40]:
dataframe_dropdup.show(10)

+--------------------+--------------------+--------------------+-----------------+--------------------+-------------+-----------------+----------------+----+--------------+--------------------+-------------+
|                 _id|  amazon_product_url|              author| bestsellers_date|         description|        price|   published_date|       publisher|rank|rank_last_week|               title|weeks_on_list|
+--------------------+--------------------+--------------------+-----------------+--------------------+-------------+-----------------+----------------+----+--------------+--------------------+-------------+
|{5b4aa4ead3089013...|http://www.amazon...|Clive Cussler wit...|{{1213401600000}}|Juan Cabrillo and...|{26.95, null}|{{1214697600000}}|          Putnam| {4}|           {3}|         PLAGUE SHIP|          {2}|
|{5b4aa4ead3089013...|http://www.amazon...|      Jeffery Deaver|{{1215820800000}}|Detectives Lincol...|    {null, 0}|{{1217116800000}}|Simon & Schuster|{20}|           

In [41]:
dataframe.show()

+--------------------+--------------------+--------------------+-----------------+--------------------+-------------+-----------------+--------------------+----+--------------+--------------------+-------------+
|                 _id|  amazon_product_url|              author| bestsellers_date|         description|        price|   published_date|           publisher|rank|rank_last_week|               title|weeks_on_list|
+--------------------+--------------------+--------------------+-----------------+--------------------+-------------+-----------------+--------------------+----+--------------+--------------------+-------------+
|{5b4aa4ead3089013...|http://www.amazon...|       Dean R Koontz|{{1211587200000}}|Odd Thomas, who c...|   {null, 27}|{{1212883200000}}|              Bantam| {1}|           {0}|           ODD HOURS|          {1}|
|{5b4aa4ead3089013...|http://www.amazon...|     Stephenie Meyer|{{1211587200000}}|Aliens have taken...|{25.99, null}|{{1212883200000}}|       Little, Br

In [43]:
#Select

#show all the entires in author column
dataframe.select('author').show(10)

#Show all the entries in author, title,rank,price column

dataframe.select("author","title","rank","price").show(10)


+--------------------+
|              author|
+--------------------+
|       Dean R Koontz|
|     Stephenie Meyer|
|        Emily Giffin|
|   Patricia Cornwell|
|     Chuck Palahniuk|
|James Patterson a...|
|       John Sandford|
|       Jimmy Buffett|
|    Elizabeth George|
|      David Baldacci|
+--------------------+
only showing top 10 rows

+--------------------+--------------------+----+-------------+
|              author|               title|rank|        price|
+--------------------+--------------------+----+-------------+
|       Dean R Koontz|           ODD HOURS| {1}|   {null, 27}|
|     Stephenie Meyer|            THE HOST| {2}|{25.99, null}|
|        Emily Giffin|LOVE THE ONE YOU'...| {3}|{24.95, null}|
|   Patricia Cornwell|           THE FRONT| {4}|{22.95, null}|
|     Chuck Palahniuk|               SNUFF| {5}|{24.95, null}|
|James Patterson a...|SUNDAYS AT TIFFANY’S| {6}|{24.99, null}|
|       John Sandford|        PHANTOM PREY| {7}|{26.95, null}|
|       Jimmy Buffett|

In [46]:
#when operation
#show title and assign 0 or 1 depending on title

dataframe.select("title", when(dataframe.title != 'ODD HOURS',1).otherwise(0)).show()

+--------------------+-----------------------------------------------------+
|               title|CASE WHEN (NOT (title = ODD HOURS)) THEN 1 ELSE 0 END|
+--------------------+-----------------------------------------------------+
|           ODD HOURS|                                                    0|
|            THE HOST|                                                    1|
|LOVE THE ONE YOU'...|                                                    1|
|           THE FRONT|                                                    1|
|               SNUFF|                                                    1|
|SUNDAYS AT TIFFANY’S|                                                    1|
|        PHANTOM PREY|                                                    1|
|          SWINE NOT?|                                                    1|
|     CARELESS IN RED|                                                    1|
|     THE WHOLE TRUTH|                                                    1|

In [48]:
#isin operation

#show rows with specified authors if in the given options

dataframe[dataframe.author.isin("John Sandford","Emily Giffin")].show(5)

+--------------------+--------------------+-------------+-----------------+--------------------+-------------+-----------------+------------+----+--------------+--------------------+-------------+
|                 _id|  amazon_product_url|       author| bestsellers_date|         description|        price|   published_date|   publisher|rank|rank_last_week|               title|weeks_on_list|
+--------------------+--------------------+-------------+-----------------+--------------------+-------------+-----------------+------------+----+--------------+--------------------+-------------+
|{5b4aa4ead3089013...|http://www.amazon...| Emily Giffin|{{1211587200000}}|A woman's happy m...|{24.95, null}|{{1212883200000}}|St. Martin's| {3}|           {2}|LOVE THE ONE YOU'...|          {2}|
|{5b4aa4ead3089013...|http://www.amazon...|John Sandford|{{1211587200000}}|The Minneapolis d...|{26.95, null}|{{1212883200000}}|      Putnam| {7}|           {4}|        PHANTOM PREY|          {3}|
|{5b4aa4ead3089

In [49]:
#Like operation---results boolean value

dataframe.select('author', 'title', dataframe.title.like("% THE %")).show(5)

+-----------------+--------------------+------------------+
|           author|               title|title LIKE % THE %|
+-----------------+--------------------+------------------+
|    Dean R Koontz|           ODD HOURS|             false|
|  Stephenie Meyer|            THE HOST|             false|
|     Emily Giffin|LOVE THE ONE YOU'...|              true|
|Patricia Cornwell|           THE FRONT|             false|
|  Chuck Palahniuk|               SNUFF|             false|
+-----------------+--------------------+------------------+
only showing top 5 rows



In [50]:
#starswith and endswith operation

dataframe.select('author','title',dataframe.title.startswith('THE')).show()

+--------------------+--------------------+----------------------+
|              author|               title|startswith(title, THE)|
+--------------------+--------------------+----------------------+
|       Dean R Koontz|           ODD HOURS|                 false|
|     Stephenie Meyer|            THE HOST|                  true|
|        Emily Giffin|LOVE THE ONE YOU'...|                 false|
|   Patricia Cornwell|           THE FRONT|                  true|
|     Chuck Palahniuk|               SNUFF|                 false|
|James Patterson a...|SUNDAYS AT TIFFANY’S|                 false|
|       John Sandford|        PHANTOM PREY|                 false|
|       Jimmy Buffett|          SWINE NOT?|                 false|
|    Elizabeth George|     CARELESS IN RED|                 false|
|      David Baldacci|     THE WHOLE TRUTH|                  true|
|        Troy Denning|          INVINCIBLE|                 false|
|          James Frey|BRIGHT SHINY MORNING|                 fa

In [54]:
dataframe.select('author','title',dataframe.title.endswith('WAVE')).show(5)

+-----------------+--------------------+---------------------+
|           author|               title|endswith(title, WAVE)|
+-----------------+--------------------+---------------------+
|    Dean R Koontz|           ODD HOURS|                false|
|  Stephenie Meyer|            THE HOST|                false|
|     Emily Giffin|LOVE THE ONE YOU'...|                false|
|Patricia Cornwell|           THE FRONT|                false|
|  Chuck Palahniuk|               SNUFF|                false|
+-----------------+--------------------+---------------------+
only showing top 5 rows



In [55]:
# Substring

dataframe.select(dataframe.author.substr(1,4).alias('title')).show(5)

+-----+
|title|
+-----+
| Dean|
| Step|
| Emil|
| Patr|
| Chuc|
+-----+
only showing top 5 rows



In [56]:
dataframe.select(dataframe.author.substr(3,7).alias('title')).show(5)

+-------+
|  title|
+-------+
|an R Ko|
|ephenie|
|ily Gif|
|tricia |
|uck Pal|
+-------+
only showing top 5 rows



In [57]:
dataframe.select(dataframe.author.substr(1,6).alias('title')).show(5)

+------+
| title|
+------+
|Dean R|
|Stephe|
|Emily |
|Patric|
|Chuck |
+------+
only showing top 5 rows



In [58]:
# Adding columns

from pyspark.sql import functions as F

dataframe =dataframe.withColumn('NewColumn', F.lit('This is a New column'))

In [59]:
display(dataframe)

DataFrame[_id: struct<$oid:string>, amazon_product_url: string, author: string, bestsellers_date: struct<$date:struct<$numberLong:string>>, description: string, price: struct<$numberDouble:string,$numberInt:string>, published_date: struct<$date:struct<$numberLong:string>>, publisher: string, rank: struct<$numberInt:string>, rank_last_week: struct<$numberInt:string>, title: string, weeks_on_list: struct<$numberInt:string>, NewColumn: string]

In [60]:
dataframe.show()

+--------------------+--------------------+--------------------+-----------------+--------------------+-------------+-----------------+--------------------+----+--------------+--------------------+-------------+--------------------+
|                 _id|  amazon_product_url|              author| bestsellers_date|         description|        price|   published_date|           publisher|rank|rank_last_week|               title|weeks_on_list|           NewColumn|
+--------------------+--------------------+--------------------+-----------------+--------------------+-------------+-----------------+--------------------+----+--------------+--------------------+-------------+--------------------+
|{5b4aa4ead3089013...|http://www.amazon...|       Dean R Koontz|{{1211587200000}}|Odd Thomas, who c...|   {null, 27}|{{1212883200000}}|              Bantam| {1}|           {0}|           ODD HOURS|          {1}|This is a New column|
|{5b4aa4ead3089013...|http://www.amazon...|     Stephenie Meyer|{{12

In [61]:
#updating the columns
dataframe =dataframe.withColumnRenamed('amazon_product_url', 'URL')

In [62]:
dataframe.show()

+--------------------+--------------------+--------------------+-----------------+--------------------+-------------+-----------------+--------------------+----+--------------+--------------------+-------------+--------------------+
|                 _id|                 URL|              author| bestsellers_date|         description|        price|   published_date|           publisher|rank|rank_last_week|               title|weeks_on_list|           NewColumn|
+--------------------+--------------------+--------------------+-----------------+--------------------+-------------+-----------------+--------------------+----+--------------+--------------------+-------------+--------------------+
|{5b4aa4ead3089013...|http://www.amazon...|       Dean R Koontz|{{1211587200000}}|Odd Thomas, who c...|   {null, 27}|{{1212883200000}}|              Bantam| {1}|           {0}|           ODD HOURS|          {1}|This is a New column|
|{5b4aa4ead3089013...|http://www.amazon...|     Stephenie Meyer|{{12

In [63]:
#Removing Columns
#drop()

dataframe_remove = dataframe.drop('publisher','published_date').show(5)

+--------------------+--------------------+-----------------+-----------------+--------------------+-------------+----+--------------+--------------------+-------------+--------------------+
|                 _id|                 URL|           author| bestsellers_date|         description|        price|rank|rank_last_week|               title|weeks_on_list|           NewColumn|
+--------------------+--------------------+-----------------+-----------------+--------------------+-------------+----+--------------+--------------------+-------------+--------------------+
|{5b4aa4ead3089013...|http://www.amazon...|    Dean R Koontz|{{1211587200000}}|Odd Thomas, who c...|   {null, 27}| {1}|           {0}|           ODD HOURS|          {1}|This is a New column|
|{5b4aa4ead3089013...|http://www.amazon...|  Stephenie Meyer|{{1211587200000}}|Aliens have taken...|{25.99, null}| {2}|           {1}|            THE HOST|          {3}|This is a New column|
|{5b4aa4ead3089013...|http://www.amazon...|  

In [65]:
dataframe_remove1 =dataframe.drop(dataframe.publisher).drop(dataframe.published_date).show(5)

+--------------------+--------------------+-----------------+-----------------+--------------------+-------------+----+--------------+--------------------+-------------+--------------------+
|                 _id|                 URL|           author| bestsellers_date|         description|        price|rank|rank_last_week|               title|weeks_on_list|           NewColumn|
+--------------------+--------------------+-----------------+-----------------+--------------------+-------------+----+--------------+--------------------+-------------+--------------------+
|{5b4aa4ead3089013...|http://www.amazon...|    Dean R Koontz|{{1211587200000}}|Odd Thomas, who c...|   {null, 27}| {1}|           {0}|           ODD HOURS|          {1}|This is a New column|
|{5b4aa4ead3089013...|http://www.amazon...|  Stephenie Meyer|{{1211587200000}}|Aliens have taken...|{25.99, null}| {2}|           {1}|            THE HOST|          {3}|This is a New column|
|{5b4aa4ead3089013...|http://www.amazon...|  

In [67]:
#GroupBy author, count the books of the authors in the list

dataframe.groupBy('author').count().show(5)



+-----------------+-----+
|           author|count|
+-----------------+-----+
|       James Frey|    2|
| Elin Hilderbrand|   58|
|Sharon Kay Penman|    2|
|      Kate Jacobs|    3|
|    Karen Robards|    6|
+-----------------+-----+
only showing top 5 rows



In [70]:
# Filter

dataframe.filter(dataframe['title']=='THE HOST').show()

+--------------------+--------------------+---------------+-----------------+--------------------+-------------+-----------------+-------------+----+--------------+--------+-------------+--------------------+
|                 _id|                 URL|         author| bestsellers_date|         description|        price|   published_date|    publisher|rank|rank_last_week|   title|weeks_on_list|           NewColumn|
+--------------------+--------------------+---------------+-----------------+--------------------+-------------+-----------------+-------------+----+--------------+--------+-------------+--------------------+
|{5b4aa4ead3089013...|http://www.amazon...|Stephenie Meyer|{{1211587200000}}|Aliens have taken...|{25.99, null}|{{1212883200000}}|Little, Brown| {2}|           {1}|THE HOST|          {3}|This is a New column|
|{5b4aa4ead3089013...|http://www.amazon...|Stephenie Meyer|{{1212192000000}}|Aliens have taken...|{25.99, null}|{{1213488000000}}|Little, Brown| {2}|           {2}|

In [71]:
# Handling missing values

dataframe.na.fill()


dataframe.dropna()


dataframe.na.replace()

TypeError: ignored

In [73]:
dataframe.rdd.getNumPartitions()

2

In [75]:
df= dataframe.repartition(20)

In [76]:
df.rdd.getNumPartitions()

20

In [77]:
## SQL Commands in Spark

In [78]:
dataframe.registerTempTable('df1')

/usr/local/lib/python3.8/dist-packages/pyspark/sql/dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [82]:
sc.sql('Select * from df1').show(3)

+--------------------+--------------------+---------------+-----------------+--------------------+-------------+-----------------+-------------+----+--------------+--------------------+-------------+--------------------+
|                 _id|                 URL|         author| bestsellers_date|         description|        price|   published_date|    publisher|rank|rank_last_week|               title|weeks_on_list|           NewColumn|
+--------------------+--------------------+---------------+-----------------+--------------------+-------------+-----------------+-------------+----+--------------+--------------------+-------------+--------------------+
|{5b4aa4ead3089013...|http://www.amazon...|  Dean R Koontz|{{1211587200000}}|Odd Thomas, who c...|   {null, 27}|{{1212883200000}}|       Bantam| {1}|           {0}|           ODD HOURS|          {1}|This is a New column|
|{5b4aa4ead3089013...|http://www.amazon...|Stephenie Meyer|{{1211587200000}}|Aliens have taken...|{25.99, null}|{{12

In [83]:
## convert a dataframe into an RDD
rdd_convert = dataframe.rdd

In [84]:
type(rdd_convert)

pyspark.rdd.RDD

In [86]:
dataframe.toJSON().take(10)

['{"_id":{"$oid":"5b4aa4ead3089013507db18b"},"URL":"http://www.amazon.com/Odd-Hours-Dean-Koontz/dp/0553807056?tag=NYTBS-20","author":"Dean R Koontz","bestsellers_date":{"$date":{"$numberLong":"1211587200000"}},"description":"Odd Thomas, who can communicate with the dead, confronts evil forces in a California coastal town.","price":{"$numberInt":"27"},"published_date":{"$date":{"$numberLong":"1212883200000"}},"publisher":"Bantam","rank":{"$numberInt":"1"},"rank_last_week":{"$numberInt":"0"},"title":"ODD HOURS","weeks_on_list":{"$numberInt":"1"},"NewColumn":"This is a New column"}',
 '{"_id":{"$oid":"5b4aa4ead3089013507db18c"},"URL":"http://www.amazon.com/The-Host-Novel-Stephenie-Meyer/dp/0316218502?tag=NYTBS-20","author":"Stephenie Meyer","bestsellers_date":{"$date":{"$numberLong":"1211587200000"}},"description":"Aliens have taken control of the minds and bodies of most humans, but one woman won’t surrender.","price":{"$numberDouble":"25.99"},"published_date":{"$date":{"$numberLong":"12

In [92]:
DATAFRAME = dataframe.toPandas()

In [93]:
type(DATAFRAME)

pandas.core.frame.DataFrame

In [94]:
DATAFRAME

,_id,URL,author,bestsellers_date,description,price,published_date,publisher,rank,rank_last_week,title,weeks_on_list,NewColumn
0,"(5b4aa4ead3089013507db18b,)",http://www.amazon.com/Odd-Hours-Dean-Koontz/dp...,Dean R Koontz,"((1211587200000,),)","Odd Thomas, who can communicate with the dead,...","(None, 27)","((1212883200000,),)",Bantam,"(1,)","(0,)",ODD HOURS,"(1,)",This is a New column
1,"(5b4aa4ead3089013507db18c,)",http://www.amazon.com/The-Host-Novel-Stephenie...,Stephenie Meyer,"((1211587200000,),)",Aliens have taken control of the minds and bod...,"(25.99, None)","((1212883200000,),)","Little, Brown","(2,)","(1,)",THE HOST,"(3,)",This is a New column
2,"(5b4aa4ead3089013507db18d,)",http://www.amazon.com/Love-Youre-With-Emily-Gi...,Emily Giffin,"((1211587200000,),)",A woman's happy marriage is shaken when she en...,"(24.95, None)","((1212883200000,),)",St. Martin's,"(3,)","(2,)",LOVE THE ONE YOU'RE WITH,"(2,)",This is a New column
3,"(5b4aa4ead3089013507db18e,)",http://www.amazon.com/The-Front-Garano-Patrici...,Patricia Cornwell,"((1211587200000,),)",A Massachusetts state investigator and his tea...,"(22.95, None)","((1212883200000,),)",Putnam,"(4,)","(0,)",THE FRONT,"(1,)",This is a New column
4,"(5b4aa4ead3089013507db18f,)",http://www.amazon.com/Snuff-Chuck-Palahniuk/dp...,Chuck Palahniuk,"((1211587200000,),)",An aging porn queens aims to cap her career by...,"(24.95, None)","((1212883200000,),)",Doubleday,"(5,)","(0,)",SNUFF,"(1,)",This is a New column
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10190,"(5b4aa4ead3089013507dd959,)",https://www.amazon.com/Clancy-Line-Sight-Jack-...,Mike Maden,"((1530921600000,),)",Jack Ryan Jr. risks his life to protect a woma...,"(None, 0)","((1532217600000,),)",Putnam,"(11,)","(6,)",TOM CLANCY LINE OF SIGHT,"(4,)",This is a New column
10191,"(5b4aa4ead3089013507dd95a,)",https://www.amazon.com/Something-Water-Novel-C...,Catherine Steadman,"((1530921600000,),)",A documentary filmmaker and an investment bank...,"(None, 0)","((1532217600000,),)",Ballantine,"(12,)","(11,)",SOMETHING IN THE WATER,"(5,)",This is a New column
10192,"(5b4aa4ead3089013507dd95b,)",https://www.amazon.com/Little-Fires-Everywhere...,Celeste Ng,"((1530921600000,),)",An artist upends a quiet town outside Cleveland.,"(None, 0)","((1532217600000,),)",Penguin Press,"(13,)","(12,)",LITTLE FIRES EVERYWHERE,"(41,)",This is a New column
10193,"(5b4aa4ead3089013507dd95c,)",https://www.amazon.com/Shelter-Place-Nora-Robe...,Nora Roberts,"((1530921600000,),)",Survivors of a mass shooting outside a mall in...,"(None, 0)","((1532217600000,),)",St. Martin's,"(14,)","(5,)",SHELTER IN PLACE,"(6,)",This is a New column


In [108]:
dataframe.rdd.getNumPartitions()

1

In [107]:
dataframe =dataframe.coalesce(1)

In [110]:
# Write & Save file in Parque format

dataframe.select('author','title','rank','description').write.save('boot/Rankings_Description.parquet')

In [112]:
#write & Save file in json format

dataframe.select('author','title').write.save("boot/Author's_Titles.json",format = 'json')

In [113]:
#write & Save file in csv format

dataframe.select('author','publisher').write.save("boot/Author's_Publisher.csv",format = 'csv')

AnalysisException: ignored